In [5]:
import pandas as pd
from collections import defaultdict

In [6]:
cd ml-latest-small/

[Errno 2] No such file or directory: 'ml-latest-small/'
/root/notebooks/Recommendation Systems/ml-latest-small


In [7]:
movies = pd.read_csv(r"movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
all_ratings = pd.read_csv(r"all_ratings.csv")
all_ratings.tail()

,Unnamed: 0,userId,movieId,rating
100732,728,672,148626,4.0
100733,729,672,150548,3.0
100734,730,672,152091,1.0
100735,731,672,159093,3.0
100736,732,672,160438,3.0


In [9]:
def load_movie_lens_data(lens_movies, ratings):
    movies={}
    for _, row in lens_movies.iterrows():
        movies[row["movieId"]] = row["title"]
    
    prefs=defaultdict(dict)
    for _, row in ratings.iterrows():
        if row["movieId"] in movies:
            prefs[row["userId"]][movies[row["movieId"]]] = float(row["rating"])
    return prefs

In [10]:
prefs = load_movie_lens_data(movies, all_ratings)

In [11]:
from math import sqrt
def sim_pearson(prefs,o1,o2):
    # Get the list of mutually rated items
    si={}
    for item in prefs[o1]:
        if item in prefs[o2]: 
            si[item]=1
    
    # Find the number of elements
    n=len(si)
    # if they are no ratings in common, return 0
    if n==0: 
        return 0
    
    # Add up all the preferences
    sum1=sum([prefs[o1][it] for it in si])
    sum2=sum([prefs[o2][it] for it in si])
    
    # Sum up the squares
    sum1Sq=sum([pow(prefs[o1][it],2) for it in si])
    sum2Sq=sum([pow(prefs[o2][it],2) for it in si])
    
    # Sum up the products
    pSum=sum([prefs[o1][it]*prefs[o2][it] for it in si])
    # Calculate Pearson score
    num=pSum-(sum1*sum2/n)
    den=sqrt((sum1Sq-pow(sum1,2)/n)*(sum2Sq-pow(sum2,2)/n))
    if den==0: 
        return 0
    
    r=num/den
    return r

def sim_distance(prefs,o1,o2):
    # Get the list of shared_items
    si={}
    for item in prefs[o1]:
        if item in prefs[o2]:
            si[item]=1

    # if they have no ratings in common, return 0
    if len(si)==0: 
        return 0
    # Add up the squares of all the differences
    sum_of_squares=sum([pow(prefs[o1][item]-prefs[o2][item],2) for item in prefs[o1] if item in prefs[o2]])
    
    return 1/(1+sum_of_squares)

def sim_jaccard(prefs,o1,o2):

    count=0
    for item in prefs[o1]:
        if item in prefs[o2]:
            count+=1

    n1=len(prefs[o1])
    n2=len(prefs[o2])
    count=float(count)

    r=float(count/(n1+n2-count))

    return r

def sim_cosine(prefs,o1,o2):
    list={}
    num=0
    den=0
    for item in prefs[o1]:
        if item in prefs[o2]:
            list[item]=1
            num+=prefs[o1][item]*prefs[o2][item]

    if num==0:
        return 0

    den= pow( sum(pow(prefs[o1][item],2) for item in list)*sum(pow(prefs[o2][item],2) for item in list) , 0.5 )
    num=float(num)

    return num/den

In [12]:
def getRecommendations(prefs,person,similarity=sim_pearson):
    totals={}
    simSums={}
    for other in prefs:
        # don't compare me to myself
        if other==person: 
            continue
        sim=similarity(prefs,person,other)
        # ignore scores of zero or lower
        if sim<=0: 
            continue
        for item in prefs[other]:
            # only score movies I haven't seen yet
            if item not in prefs[person] or prefs[person][item]==0:
                # Similarity * Score
                totals.setdefault(item,0)
                totals[item]+=prefs[other][item]*sim
                # Sum of similarities
                simSums.setdefault(item,0)
                simSums[item]+=sim
    
    # Create the normalized list
    rankings=[(total/simSums[item],item) for item,total in totals.items( )]
    # Return the sorted list
    rankings.sort( )
    rankings.reverse( )
    return rankings

In [13]:
getRecommendations(prefs, 672)[0:10]

[(5.000000000000001, 'Wrong Cops (2013)'),
 (5.000000000000001, 'Wrong (2012)'),
 (5.000000000000001, 'Waiter (Ober) (2006)'),
 (5.000000000000001, 'Voyeur (Abel) (1986)'),
 (5.000000000000001,
  'Through the Olive Trees (Zire darakhatan zeyton) (1994)'),
 (5.000000000000001, 'The Last Days of Emma Blank (2009)'),
 (5.000000000000001, 'The Dress (1996)'),
 (5.000000000000001, 'Schneider vs. Bax (2015)'),
 (5.000000000000001, 'Reality (2014)'),
 (5.000000000000001, 'Patience Stone, The (2012)')]

In [14]:
len(prefs[672])

733

In [15]:
y = prefs[672]
for z in y.keys():
    print(z,":",y[z])

Toy Story (1995) : 3.0
Jumanji (1995) : 2.5
Heat (1995) : 4.0
GoldenEye (1995) : 4.0
American President, The (1995) : 2.0
Casino (1995) : 4.0
Sense and Sensibility (1995) : 2.5
Copycat (1995) : 2.5
Leaving Las Vegas (1995) : 3.0
Dangerous Minds (1995) : 3.0
Twelve Monkeys (a.k.a. 12 Monkeys) (1995) : 4.0
Clueless (1995) : 1.0
Seven (a.k.a. Se7en) (1995) : 5.0
Pocahontas (1995) : 0.5
Usual Suspects, The (1995) : 4.5
Mr. Holland's Opus (1995) : 3.0
Broken Arrow (1996) : 2.5
Happy Gilmore (1996) : 2.5
Braveheart (1995) : 3.5
Bad Boys (1995) : 3.0
Apollo 13 (1995) : 4.0
Canadian Bacon (1995) : 2.0
Crimson Tide (1995) : 4.5
Desperado (1995) : 2.0
Die Hard: With a Vengeance (1995) : 3.0
Net, The (1995) : 2.5
Species (1995) : 2.0
Strange Days (1995) : 2.5
Under Siege 2: Dark Territory (1995) : 2.5
Waterworld (1995) : 2.5
Clerks (1994) : 3.0
Dumb & Dumber (Dumb and Dumber) (1994) : 1.5
Interview with the Vampire: The Vampire Chronicles (1994) : 2.5
Junior (1994) : 1.0
Star Wars: Episode IV - A

Limitless (2011) : 2.5
Unknown (2011) : 3.0
Source Code (2011) : 3.0
Senna (2010) : 5.0
Thor (2011) : 3.0
Tree of Life, The (2011) : 3.0
Hangover Part II, The (2011) : 3.0
Super 8 (2011) : 3.0
Contagion (2011) : 3.0
Moneyball (2011) : 4.0
Avengers, The (2012) : 2.0
Tinker Tailor Soldier Spy (2011) : 4.0
Warrior (2011) : 3.0
Ides of March, The (2011) : 3.5
The Artist (2011) : 5.0
Headhunters (Hodejegerne) (2011) : 4.0
Expendables 2, The (2012) : 2.0
The Hunger Games (2012) : 3.0
Dark Knight Rises, The (2012) : 3.0
Bourne Legacy, The (2012) : 2.5
Sherlock Holmes: A Game of Shadows (2011) : 3.5
Intouchables (2011) : 4.5
Safe House (2012) : 2.5
John Carter (2012) : 2.5
21 Jump Street (2012) : 3.0
American Reunion (American Pie 4) (2012) : 2.0
Prometheus (2012) : 3.0
Skyfall (2012) : 4.0
Looper (2012) : 4.0
Master, The (2012) : 2.0
Taken 2 (2012) : 3.5
Argo (2012) : 4.0
Cloud Atlas (2012) : 3.0
Silver Linings Playbook (2012) : 4.5
Flight (2012) : 2.5
Batman: The Dark Knight Returns, Part 1 

In [16]:
getRecommendations(prefs, 672,similarity=sim_cosine)[0:10]

[(5.000000000000001, 'Z Channel: A Magnificent Obsession (2004)'),
 (5.000000000000001, 'World of Tomorrow (2015)'),
 (5.000000000000001, 'Wild Zero (2000)'),
 (5.000000000000001, 'Walker (1987)'),
 (5.000000000000001, 'Victoria (2015)'),
 (5.000000000000001, 'Unvanquished, The (Aparajito) (1957)'),
 (5.000000000000001, 'Two Escobars, The (2010)'),
 (5.000000000000001, 'Totally F***ed Up (1993)'),
 (5.000000000000001, 'Thief of Bagdad, The (1924)'),
 (5.000000000000001, 'The Car (1977)')]

In [17]:
def getRecommendedItems(prefs,itemMatch,user):
    userRatings=prefs[user]
    scores={}
    totalSim={}

    # Loop over items rated by this user
    for (item,rating) in userRatings.items():
        # Loop over items similar to this one
        for (similarity,item2) in itemMatch[item]:
            # Ignore if this user has already rated this item
            if item2 in userRatings: continue
            # Weighted sum of rating times similarity
            scores.setdefault(item2,0)
            scores[item2]+=similarity*rating
            # Sum of all the similarities
            totalSim.setdefault(item2,0)
            totalSim[item2]+=similarity
    # Divide each total score by total weighting to get an average
    rankings=[(round(score/totalSim[item],1),item) for item,score in scores.items( ) if totalSim[item] !=0]
    # Return the rankings from highest to lowest
    rankings.sort( )
    rankings.reverse( )
    return rankings

def calculateSimilarItems(prefs,n=10):
    # Create a dictionary of items showing which other items they
    # are most similar to.
    result={}
    # Invert the preference matrix to be item-centric
    itemPrefs=transformPrefs(prefs)
    c=0
    for item in itemPrefs:
        # Status updates for large datasets
        c+=1
        if c%1000==0: print("%d / %d" % (c,len(itemPrefs)))
        # Find the most similar items to this one
        scores=topMatches(itemPrefs,item,n=n,similarity=sim_pearson)
        result[item]=scores
    return result

def transformPrefs(prefs):
    result={}
    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item,{})
            # Flip item and person
            result[item][person]=prefs[person][item]
    return result

# Returns the best matches for person from the prefs dictionary.
# Number of results and similarity function are optional params.
def topMatches(prefs,person,n=5,similarity=sim_pearson):
    scores=[(similarity(prefs,person,other),other) for other in prefs if other!=person]
    # Sort the list so the highest scores appear at the top
    scores.sort( )
    scores.reverse( )
    return scores[0:n]

In [18]:
for user in prefs:
    if user == 15.0:
        print("user: ",user)
        ratings = prefs[user]
        for a, b in ratings.items():
            print("*",a,":",b)

user:  15.0
* Toy Story (1995) : 2.0
* Jumanji (1995) : 2.0
* Father of the Bride Part II (1995) : 4.5
* Heat (1995) : 4.0
* GoldenEye (1995) : 3.0
* American President, The (1995) : 2.5
* Nixon (1995) : 2.5
* Casino (1995) : 3.5
* Sense and Sensibility (1995) : 3.0
* Ace Ventura: When Nature Calls (1995) : 1.0
* Get Shorty (1995) : 4.5
* Copycat (1995) : 2.5
* Leaving Las Vegas (1995) : 3.0
* Twelve Monkeys (a.k.a. 12 Monkeys) (1995) : 4.0
* Babe (1995) : 3.0
* Dead Man Walking (1995) : 1.0
* Clueless (1995) : 2.5
* Mortal Kombat (1995) : 3.0
* Seven (a.k.a. Se7en) (1995) : 5.0
* Usual Suspects, The (1995) : 5.0
* Mighty Aphrodite (1995) : 2.5
* Mr. Holland's Opus (1995) : 2.0
* From Dusk Till Dawn (1996) : 0.5
* Antonia's Line (Antonia) (1995) : 5.0
* Beautiful Girls (1996) : 3.0
* Broken Arrow (1996) : 1.5
* Bottle Rocket (1996) : 4.0
* Happy Gilmore (1996) : 1.0
* Muppet Treasure Island (1996) : 2.0
* Braveheart (1995) : 3.0
* Taxi Driver (1976) : 5.0
* Rumble in the Bronx (Hont fa

* Henry Fool (1997) : 3.5
* X-Files: Fight the Future, The (1998) : 2.0
* Out of Sight (1998) : 5.0
* Smoke Signals (1998) : 3.5
* Armageddon (1998) : 2.0
* Pi (1998) : 4.0
* There's Something About Mary (1998) : 4.5
* On the Waterfront (1954) : 4.0
* In the Heat of the Night (1967) : 3.0
* Midnight Cowboy (1969) : 3.0
* French Connection, The (1971) : 3.0
* Rocky (1976) : 3.0
* Kramer vs. Kramer (1979) : 4.5
* Ordinary People (1980) : 4.0
* Rain Man (1988) : 3.5
* Driving Miss Daisy (1989) : 3.0
* Klute (1971) : 3.0
* Breakfast Club, The (1985) : 4.0
* Poltergeist (1982) : 2.5
* Exorcist, The (1973) : 5.0
* Lethal Weapon (1987) : 3.0
* Gremlins (1984) : 3.0
* Goonies, The (1985) : 4.0
* Mask of Zorro, The (1998) : 4.0
* Metropolis (1927) : 3.0
* Back to the Future Part II (1989) : 3.0
* Back to the Future Part III (1990) : 1.0
* Bambi (1942) : 2.0
* Seven Samurai (Shichinin no samurai) (1954) : 4.5
* Dangerous Liaisons (1988) : 2.5
* Lolita (1997) : 2.0
* Saving Private Ryan (1998) : 

* Proof of Life (2000) : 1.0
* Vertical Limit (2000) : 3.0
* Living Daylights, The (1987) : 3.5
* Transformers: The Movie (1986) : 3.0
* Wall Street (1987) : 3.5
* Brewster's Millions (1985) : 2.0
* Snatch (2000) : 1.0
* Chocolat (2000) : 2.0
* Dude, Where's My Car? (2000) : 5.0
* Pollock (2000) : 4.0
* What Women Want (2000) : 1.0
* Finding Forrester (2000) : 2.0
* Gift, The (2000) : 3.0
* Before Night Falls (2000) : 3.0
* Cast Away (2000) : 3.0
* Family Man, The (2000) : 2.0
* Miss Congeniality (2000) : 1.0
* O Brother, Where Art Thou? (2000) : 3.0
* State and Main (2000) : 4.0
* Dracula 2000 (2000) : 2.0
* Thirteen Days (2000) : 4.0
* Traffic (2000) : 4.0
* Shadow of the Vampire (2000) : 1.0
* House of Games (1987) : 3.0
* Antitrust (2001) : 1.0
* Panic (2000) : 1.0
* Pledge, The (2001) : 5.0
* I'm Gonna Git You Sucka (1988) : 4.0
* Amazon Women on the Moon (1987) : 4.0
* Barfly (1987) : 1.0
* Beverly Hills Cop (1984) : 3.0
* Innerspace (1987) : 3.0
* In the Mood For Love (Fa yeung 

* Be Kind Rewind (2008) : 2.5
* I Am Legend (2007) : 1.0
* Juno (2007) : 2.0
* Helvetica (2007) : 4.0
* National Treasure: Book of Secrets (2007) : 3.0
* There Will Be Blood (2007) : 1.0
* Cloverfield (2008) : 1.5
* Hellboy II: The Golden Army (2008) : 4.0
* In Bruges (2008) : 1.5
* Jumper (2008) : 1.0
* Bank Job, The (2008) : 1.5
* Dark Knight, The (2008) : 4.5
* Forgetting Sarah Marshall (2008) : 1.5
* Religulous (2008) : 4.0
* Iron Man (2008) : 4.5
* Taken (2008) : 2.5
* Reprise (2006) : 3.5
* Indiana Jones and the Kingdom of the Crystal Skull (2008) : 1.0
* Kung Fu Panda (2008) : 3.5
* You Don't Mess with the Zohan (2008) : 1.5
* Happening, The (2008) : 0.5
* Incredible Hulk, The (2008) : 3.0
* WALL·E (2008) : 4.5
* Wanted (2008) : 1.0
* Hancock (2008) : 1.0
* Get Smart (2008) : 1.0
* Up the Yangtze (2007) : 3.5
* Watchmen (2009) : 4.0
* Man on Wire (2008) : 3.5
* Pineapple Express (2008) : 1.0
* Tropic Thunder (2008) : 2.0
* Burn After Reading (2008) : 1.0
* Onion Movie, The (2008

In [19]:
for user in prefs:
    ratings = prefs[user]
    print("user: ",user,":","number: ", len(ratings))

user:  1.0 : number:  20
user:  2.0 : number:  76
user:  3.0 : number:  51
user:  4.0 : number:  204
user:  5.0 : number:  100
user:  6.0 : number:  44
user:  7.0 : number:  88
user:  8.0 : number:  116
user:  9.0 : number:  45
user:  10.0 : number:  46
user:  11.0 : number:  38
user:  12.0 : number:  61
user:  13.0 : number:  53
user:  14.0 : number:  20
user:  15.0 : number:  1700
user:  16.0 : number:  29
user:  17.0 : number:  363
user:  18.0 : number:  51
user:  19.0 : number:  423
user:  20.0 : number:  98
user:  21.0 : number:  162
user:  22.0 : number:  220
user:  23.0 : number:  726
user:  24.0 : number:  21
user:  25.0 : number:  26
user:  26.0 : number:  172
user:  27.0 : number:  23
user:  28.0 : number:  50
user:  29.0 : number:  22
user:  30.0 : number:  1011
user:  31.0 : number:  69
user:  32.0 : number:  48
user:  33.0 : number:  138
user:  34.0 : number:  187
user:  35.0 : number:  20
user:  36.0 : number:  104
user:  37.0 : number:  32
user:  38.0 : number:  111
user

user:  363.0 : number:  201
user:  364.0 : number:  36
user:  365.0 : number:  70
user:  366.0 : number:  34
user:  367.0 : number:  217
user:  368.0 : number:  21
user:  369.0 : number:  51
user:  370.0 : number:  100
user:  371.0 : number:  161
user:  372.0 : number:  33
user:  373.0 : number:  219
user:  374.0 : number:  33
user:  375.0 : number:  25
user:  376.0 : number:  59
user:  377.0 : number:  25
user:  378.0 : number:  147
user:  379.0 : number:  71
user:  380.0 : number:  1063
user:  381.0 : number:  110
user:  382.0 : number:  314
user:  383.0 : number:  62
user:  384.0 : number:  485
user:  385.0 : number:  99
user:  386.0 : number:  121
user:  387.0 : number:  279
user:  388.0 : number:  792
user:  389.0 : number:  51
user:  390.0 : number:  153
user:  391.0 : number:  113
user:  392.0 : number:  25
user:  393.0 : number:  51
user:  394.0 : number:  205
user:  395.0 : number:  23
user:  396.0 : number:  141
user:  397.0 : number:  59
user:  398.0 : number:  53
user:  399

In [20]:
itemsim=calculateSimilarItems(prefs,n=50)

1000 / 9064
2000 / 9064
3000 / 9064
4000 / 9064
5000 / 9064
6000 / 9064
7000 / 9064
8000 / 9064
9000 / 9064


In [21]:
itemPrefs=transformPrefs(prefs)
print(len(itemPrefs))
for item in itemPrefs:
    ratings = itemPrefs[item]
    print("Movie:",item,":","Ratings:", len(ratings))

9064
Movie: Dangerous Minds (1995) : Ratings: 43
Movie: Dumbo (1941) : Ratings: 42
Movie: Sleepers (1996) : Ratings: 33
Movie: Escape from New York (1981) : Ratings: 48
Movie: Cinema Paradiso (Nuovo cinema Paradiso) (1989) : Ratings: 46
Movie: Deer Hunter, The (1978) : Ratings: 49
Movie: Ben-Hur (1959) : Ratings: 47
Movie: Gandhi (1982) : Ratings: 46
Movie: Dracula (Bram Stoker's Dracula) (1992) : Ratings: 52
Movie: Cape Fear (1991) : Ratings: 39
Movie: Star Trek: The Motion Picture (1979) : Ratings: 47
Movie: Beavis and Butt-Head Do America (1996) : Ratings: 47
Movie: French Connection, The (1971) : Ratings: 47
Movie: Tron (1982) : Ratings: 47
Movie: Gods Must Be Crazy, The (1980) : Ratings: 36
Movie: Willow (1988) : Ratings: 42
Movie: Antz (1998) : Ratings: 53
Movie: Fly, The (1986) : Ratings: 48
Movie: Time Bandits (1981) : Ratings: 43
Movie: Blazing Saddles (1974) : Ratings: 62
Movie: GoldenEye (1995) : Ratings: 123
Movie: Sense and Sensibility (1995) : Ratings: 87
Movie: Clueless 

Movie: Bend It Like Beckham (2002) : Ratings: 42
Movie: Bruce Almighty (2003) : Ratings: 51
Movie: 28 Days Later (2002) : Ratings: 52
Movie: Lost in Translation (2003) : Ratings: 86
Movie: Love Actually (2003) : Ratings: 56
Movie: Napoleon Dynamite (2004) : Ratings: 45
Movie: Super Size Me (2004) : Ratings: 46
Movie: I, Robot (2004) : Ratings: 58
Movie: Million Dollar Baby (2004) : Ratings: 65
Movie: Hotel Rwanda (2004) : Ratings: 32
Movie: Charlie and the Chocolate Factory (2005) : Ratings: 47
Movie: Crash (2004) : Ratings: 48
Movie: Mr. & Mrs. Smith (2005) : Ratings: 50
Movie: Wedding Crashers (2005) : Ratings: 42
Movie: 40-Year-Old Virgin, The (2005) : Ratings: 52
Movie: Walk the Line (2005) : Ratings: 43
Movie: Chronicles of Narnia: The Lion, the Witch and the Wardrobe, The (2005) : Ratings: 49
Movie: King Kong (2005) : Ratings: 23
Movie: Borat: Cultural Learnings of America for Make Benefit Glorious Nation of Kazakhstan (2006) : Ratings: 45
Movie: Taxi Driver (1976) : Ratings: 118

Movie: Dolores Claiborne (1995) : Ratings: 25
Movie: Eat Drink Man Woman (Yin shi nan nu) (1994) : Ratings: 24
Movie: Exotica (1994) : Ratings: 13
Movie: Forget Paris (1995) : Ratings: 34
Movie: Hoop Dreams (1994) : Ratings: 61
Movie: I.Q. (1994) : Ratings: 41
Movie: Natural Born Killers (1994) : Ratings: 108
Movie: Three Colors: Red (Trois couleurs: Rouge) (1994) : Ratings: 32
Movie: Three Colors: Blue (Trois couleurs: Bleu) (1993) : Ratings: 31
Movie: Three Colors: White (Trzy kolory: Bialy) (1994) : Ratings: 22
Movie: Swimming with Sharks (1995) : Ratings: 14
Movie: Underneath (1995) : Ratings: 3
Movie: Muriel's Wedding (1994) : Ratings: 56
Movie: Crow, The (1994) : Ratings: 67
Movie: Red Rock West (1992) : Ratings: 18
Movie: Cabin Boy (1994) : Ratings: 9
Movie: Dazed and Confused (1993) : Ratings: 37
Movie: Hot Shots! Part Deux (1993) : Ratings: 60
Movie: Hudsucker Proxy, The (1994) : Ratings: 49
Movie: Kalifornia (1993) : Ratings: 32
Movie: King of the Hill (1993) : Ratings: 4
Mov

Movie: Blue Velvet (1986) : Ratings: 41
Movie: Snake Eyes (1998) : Ratings: 23
Movie: Adventures in Babysitting (1987) : Ratings: 22
Movie: Weird Science (1985) : Ratings: 37
Movie: Pretty in Pink (1986) : Ratings: 42
Movie: Rosemary's Baby (1968) : Ratings: 36
Movie: Blade (1998) : Ratings: 58
Movie: Strangers on a Train (1951) : Ratings: 23
Movie: Rounders (1998) : Ratings: 36
Movie: Cube (1997) : Ratings: 21
Movie: Broadcast News (1987) : Ratings: 36
Movie: Indecent Proposal (1993) : Ratings: 15
Movie: Pecker (1998) : Ratings: 14
Movie: Thing, The (1982) : Ratings: 33
Movie: Nashville (1975) : Ratings: 21
Movie: 2010: The Year We Make Contact (1984) : Ratings: 15
Movie: Elephant Man, The (1980) : Ratings: 28
Movie: Pleasantville (1998) : Ratings: 64
Movie: Gods and Monsters (1998) : Ratings: 23
Movie: Siege, The (1998) : Ratings: 16
Movie: Elizabeth (1998) : Ratings: 48
Movie: Meet Joe Black (1998) : Ratings: 23
Movie: Nights of Cabiria (Notti di Cabiria, Le) (1957) : Ratings: 7
Mov

Movie: But I'm a Cheerleader (1999) : Ratings: 5
Movie: Shower (Xizao) (1999) : Ratings: 2
Movie: Blow-Up (Blowup) (1966) : Ratings: 15
Movie: Criminal Lovers (1999) : Ratings: 1
Movie: Wonderland (1999) : Ratings: 4
Movie: Hollow Man (2000) : Ratings: 30
Movie: Tao of Steve, The (2000) : Ratings: 12
Movie: Aimée & Jaguar (1999) : Ratings: 2
Movie: Naked Gun: From the Files of Police Squad!, The (1988) : Ratings: 35
Movie: Bring It On (2000) : Ratings: 20
Movie: Anatomy (Anatomie) (2000) : Ratings: 2
Movie: Nurse Betty (2000) : Ratings: 21
Movie: Way of the Gun, The (2000) : Ratings: 8
Movie: Dancer in the Dark (2000) : Ratings: 23
Movie: Best in Show (2000) : Ratings: 59
Movie: Girlfight (2000) : Ratings: 3
Movie: Bamboozled (2000) : Ratings: 3
Movie: Contender, The (2000) : Ratings: 17
Movie: Lost Souls (2000) : Ratings: 2
Movie: Billy Elliot (2000) : Ratings: 44
Movie: Bedazzled (2000) : Ratings: 18
Movie: Pay It Forward (2000) : Ratings: 23
Movie: Charlie's Angels (2000) : Ratings:

...

Movie: Russians Are Coming, the Russians Are Coming, The (1966) : Ratings: 3
Movie: The Emperor's Club (2002) : Ratings: 1
Movie: Little Romance, A (1979) : Ratings: 2
Movie: Firefox (1982) : Ratings: 2
Movie: Dennis the Menace (1993) : Ratings: 2
Movie: Patch of Blue, A (1965) : Ratings: 3
Movie: Hunted, The (2003) : Ratings: 2
Movie: Wind (1992) : Ratings: 1
Movie: Treasure Island (1950) : Ratings: 2
Movie: Far Country, The (1954) : Ratings: 1
Movie: Duel at Diablo (1966) : Ratings: 1
Movie: Unforgiven, The (1960) : Ratings: 5
Movie: Flight of the Intruder (1991) : Ratings: 1
Movie: My Side of the Mountain (1969) : Ratings: 1
Movie: Thin Man Goes Home, The (1945) : Ratings: 1
Movie: After the Thin Man (1936) : Ratings: 3
Movie: Song of the Thin Man (1947) : Ratings: 2
Movie: Day at the Races, A (1937) : Ratings: 7
Movie: All of Me (1984) : Ratings: 6
Movie: Enemy Below, The (1957) : Ratings: 1
Movie: Z (1969) : Ratings: 3
Movie: Decline of the American Empire, The (Déclin de l'empire

Movie: Sound and Fury (2000) : Ratings: 1
Movie: Saints and Soldiers (2003) : Ratings: 1
Movie: Vincent & Theo (1990) : Ratings: 1
Movie: Beyond the Sea (2004) : Ratings: 1
Movie: In the Realms of the Unreal (2004) : Ratings: 1
Movie: Lessons of Darkness (Lektionen in Finsternis) (1992) : Ratings: 1
Movie: Fata Morgana (1971) : Ratings: 1
Movie: Bunny Lake Is Missing (1965) : Ratings: 2
Movie: Night and the City (1950) : Ratings: 1
Movie: Twentieth Century (1934) : Ratings: 1
Movie: Call Northside 777 (1948) : Ratings: 2
Movie: Sorrow and the Pity, The (Le chagrin et la pitié) (1969) : Ratings: 3
Movie: Big Store, The (1941) : Ratings: 2
Movie: Cocoanuts, The (1929) : Ratings: 1
Movie: 25 Watts (2001) : Ratings: 1
Movie: Prime Cut (1972) : Ratings: 1
Movie: In Old Chicago (1937) : Ratings: 1
Movie: Anderson Tapes, The (1971) : Ratings: 1
Movie: No Direction Home: Bob Dylan (2005) : Ratings: 3
Movie: Sounder (1972) : Ratings: 3
Movie: Richard Pryor: Live in Concert (1979) : Ratings: 2
M

Movie: Shape of Things, The (2003) : Ratings: 2
Movie: Car Wash (1976) : Ratings: 2
Movie: Start the Revolution Without Me (1970) : Ratings: 1
Movie: Double Life, A (1947) : Ratings: 1
Movie: House of Wax (1953) : Ratings: 1
Movie: Angel at My Table, An (1990) : Ratings: 1
Movie: Incident at Oglala (1992) : Ratings: 1
Movie: Death in Venice (Morte a Venezia) (1971) : Ratings: 1
Movie: Ugly American, The (1963) : Ratings: 1
Movie: Gas, Food, Lodging (1992) : Ratings: 1
Movie: That Was Then... This Is Now (1985) : Ratings: 1
Movie: Dance with a Stranger (1985) : Ratings: 1
Movie: Guarding Tess (1994) : Ratings: 1
Movie: Proof (1991) : Ratings: 1
Movie: Wedding Banquet, The (Xi yan) (1993) : Ratings: 1
Movie: Juliet of the Spirits (Giulietta degli spiriti) (1965) : Ratings: 1
Movie: Sweet Dreams (1985) : Ratings: 1
Movie: Ryan's Daughter (1970) : Ratings: 1
Movie: Spring Forward (1999) : Ratings: 1
Movie: Killing of a Chinese Bookie, The (1976) : Ratings: 1
Movie: Darling (1965) : Ratings

Movie: Ugly, The (1997) : Ratings: 1
Movie: Friday the 13th Part VII: The New Blood (1988) : Ratings: 1
Movie: Halloween 5: The Revenge of Michael Myers (1989) : Ratings: 2
Movie: Prom Night III: The Last Kiss (1989) : Ratings: 1
Movie: Prom Night IV: Deliver Us From Evil (1992) : Ratings: 1
Movie: Gnome-Mobile, The (1967) : Ratings: 1
Movie: Hot Lead and Cold Feet (1978) : Ratings: 1
Movie: I Married A Strange Person! (1997) : Ratings: 1
Movie: Rich and Strange (1931) : Ratings: 1
Movie: Parasite (1982) : Ratings: 1
Movie: Impostors, The (1998) : Ratings: 2
Movie: Heart Condition (1990) : Ratings: 1
Movie: Gloria (1999) : Ratings: 1
Movie: Children of the Corn II: The Final Sacrifice (1993) : Ratings: 1
Movie: House on Haunted Hill (1959) : Ratings: 1
Movie: Avalanche (1978) : Ratings: 1
Movie: Goodbye Lover (1999) : Ratings: 1
Movie: Howling II: Your Sister Is a Werewolf (1985) : Ratings: 1
Movie: Deadtime Stories (1987) : Ratings: 1
Movie: Adventures of Sebastian Cole, The (1998) : 

Movie: Shadow Dancer (2012) : Ratings: 1
Movie: Why Stop Now (2012) : Ratings: 1
Movie: Liability, The (2012) : Ratings: 1
Movie: Stand Up Guys (2012) : Ratings: 1
Movie: Identity Thief (2013) : Ratings: 2
Movie: Incredible Burt Wonderstone, The (2013) : Ratings: 1
Movie: Call, The (2013) : Ratings: 1
Movie: First Time, The (2012) : Ratings: 2
Movie: Brass Teapot, The (2012) : Ratings: 1
Movie: Dark Tide (2012) : Ratings: 1
Movie: English Teacher, The (2013) : Ratings: 1
Movie: Pawn (2013) : Ratings: 1
Movie: Syrup (2013) : Ratings: 1
Movie: Not Suitable for Children (2012) : Ratings: 1
Movie: Knot, The (2012) : Ratings: 1
Movie: Lifeguard, The (2013) : Ratings: 1
Movie: Sharknado (2013) : Ratings: 1
Movie: Turbo (2013) : Ratings: 1
Movie: Coffee Town (2013) : Ratings: 1
Movie: Revenge for Jolly! (2012) : Ratings: 1
Movie: Smurfs 2, The (2013) : Ratings: 1
Movie: Alan Partridge: Alpha Papa (2013) : Ratings: 1
Movie: Man of Tai Chi (2013) : Ratings: 1
Movie: Planes (2013) : Ratings: 1
M

In [22]:
item = "Spice World (1997)"
ratings = itemPrefs[item]
scores=[(sim_pearson(itemPrefs,item,other),other) for other in itemPrefs if other!=item]
# Sort the list so the highest scores appear at the top
scores.sort( )
scores.reverse( )
scores

[(1.000000000000004, 'Witness (1985)'),
 (1.000000000000004, 'Sideways (2004)'),
 (1.000000000000004, 'Cool Runnings (1993)'),
 (1.000000000000004, 'Amadeus (1984)'),
 (1.0000000000000033, 'Spirited Away (Sen to Chihiro no kamikakushi) (2001)'),
 (1.000000000000001, 'History of the World: Part I (1981)'),
 (1.0000000000000007, 'Benny & Joon (1993)'),
 (1.0000000000000004, 'Say Anything... (1989)'),
 (1.0000000000000004, 'French Connection, The (1971)'),
 (1.0, 'Zack and Miri Make a Porno (2008)'),
 (1.0, 'You Will Meet a Tall Dark Stranger (2010)'),
 (1.0, 'X2: X-Men United (2003)'),
 (1.0, 'Winnie the Pooh and the Blustery Day (1968)'),
 (1.0, 'What Lies Beneath (2000)'),
 (1.0, 'Wedding Planner, The (2001)'),
 (1.0, 'Wedding Crashers (2005)'),
 (1.0, 'War of the Worlds (2005)'),
 (1.0, 'Waiting to Exhale (1995)'),
 (1.0, 'Vertical Limit (2000)'),
 (1.0, 'Van Helsing (2004)'),
 (1.0, 'Up Close and Personal (1996)'),
 (1.0, 'U.S. Marshals (1998)'),
 (1.0, 'Tuxedo, The (2002)'),
 (1.0, 

In [23]:
item = "Spice World (1997)"
print(itemPrefs[item])
item = "Zack and Miri Make a Porno (2008)"
print(itemPrefs[item])

{33.0: 1.0, 73.0: 0.5, 111.0: 2.5, 128.0: 3.0, 176.0: 0.5, 285.0: 3.0, 321.0: 3.0, 363.0: 2.0, 412.0: 2.0, 472.0: 1.0, 518.0: 1.0, 582.0: 3.0, 624.0: 1.0}
{15.0: 3.5, 73.0: 3.5, 187.0: 4.0, 250.0: 3.0, 297.0: 3.0, 299.0: 3.5, 380.0: 3.5, 457.0: 2.5, 486.0: 3.0, 574.0: 4.0, 612.0: 2.5, 615.0: 2.0, 624.0: 4.5}


In [24]:
p1 = "Spice World (1997)"
p2 = "Zack and Miri Make a Porno (2008)"
si={}
for item in itemPrefs[p1]:
    if item in itemPrefs[p2]: 
        si[item]=1

x=[itemPrefs[p1][it] for it in si]
y=[itemPrefs[p2][it] for it in si]
print(x)
print(y)
plt.scatter(x, y)
plt.show()

[0.5, 1.0]
[3.5, 4.5]


NameError: name 'plt' is not defined

In [ ]:
%matplotlib inline
x, y = list(itemPrefs["Spice World (1997)"].keys()), list(itemPrefs["Spice World (1997)"].values())
import seaborn as sns;
sns.set()
import matplotlib as mpl
mpl.rcParams['font.family'] = 'serif'
import matplotlib.pyplot as plt
#fig, ax = plt.subplots(figsize=(16, 9))  # lets do wide screen ratio
#ax.plot(x,y);
plt.scatter(x, y)
plt.show()

In [ ]:
%matplotlib inline
x, y = list(itemPrefs["Zack and Miri Make a Porno (2008)"].keys()), list(itemPrefs["Zack and Miri Make a Porno (2008)"].values())
import seaborn as sns;
sns.set()
import matplotlib as mpl
mpl.rcParams['font.family'] = 'serif'
import matplotlib.pyplot as plt
#fig, ax = plt.subplots(figsize=(16, 9))  # lets do wide screen ratio
#ax.plot(x,y);
plt.scatter(x, y)
plt.show()

In [ ]:
getRecommendedItems(prefs,itemsim,672)[0:30]

In [ ]:
userRatings=x[251276]
scores={}
totalSim={}

# Loop over items rated by this user
for (item,rating) in userRatings.items():
    #if item != 'The Blair Witch Project':
    #    continue
    print(item,":", rating)
    # Loop over items similar to this one
    for (similarity,item2) in itemsim[item]:
        # Ignore if this user has already rated this item
        if item2 in userRatings: 
            continue
        print("*",round(similarity,1),":", item2)
        # Weighted sum of rating times similarity
        scores.setdefault(item2,0)
        scores[item2]+=similarity*rating
        # Sum of all the similarities
        totalSim.setdefault(item2,0)
        totalSim[item2]+=similarity

#for item,score in scores.items():
#    print('$',item,":", score,":",totalSim[item])
#    print('$$',item,":", score/totalSim[item])
        
# Divide each total score by total weighting to get an average
rankings=[(round(score/totalSim[item],1),item) for item,score in scores.items() if totalSim[item] !=0]

rankings.sort( )
rankings.reverse( )